In [2]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from google import genai
import json
from pydantic import BaseModel
import time
import pymupdf
import tempfile

In [3]:
anexos = pd.read_excel('docs/jusbrasil.xlsx', sheet_name="Anexos")

In [4]:
anexos_copia = anexos.copy().loc[:,["processoID", "processoAnexoID", "Download copia", "Tipo de anexo", "Publicado em"]]
sentencas = anexos_copia.loc[anexos["Tipo de anexo"].isin(["SENTENCA"]),
                                           ["processoID", "processoAnexoID", "Download copia", "Publicado em"]]
sentencas.head()

,processoID,processoAnexoID,Download copia,Publicado em
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,2021-08-13
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,2022-05-16
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,2022-07-20
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,2024-07-31
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,2025-01-29


In [5]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto)
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = sentencas[sentencas['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)
acessos_negados.head()

,processoID,processoAnexoID,Download copia,Publicado em
1293,471515190,727395468,http://jud-anexos.digesto.com.br/3a9fbbe2c4f69...,2022-04-28
1302,471515190,727395467,http://jud-anexos.digesto.com.br/7767cbbc02e35...,2024-03-07
2876,595998611,1403409515,http://jud-anexos.digesto.com.br/0ec3e0c015d72...,2024-08-05
2877,595998611,1403409516,http://jud-anexos.digesto.com.br/385306277d658...,2024-08-05
8000,690025576,2015456650,http://jud-anexos.digesto.com.br/fb01c9fd839aa...,2021-01-01


In [ ]:
# Dropando os acessos negados
sentencas_acessaveis = sentencas[sentencas["processoAnexoID"].isin(acessos_negados["processoAnexoID"]) == False]

# Exportando sentenças sem acesso negado para um arquivo Excel
# sentencas_acessaveis.to_excel("docs/sentencas_acessaveis.xlsx", index=False)

In [7]:
# Verificando quantidade de processos após o drop
sentencas_acessaveis["processoID"].unique().shape

(223,)

In [8]:
ids_processos = sentencas_acessaveis["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

Número de processos com mais de uma sentença: 84


In [13]:
sentencas_final = sentencas_acessaveis.copy()
for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas_acessaveis['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        if linhas_correspondentes['Publicado em'].nunique() == 1:
            # Data de publicação igual: eliminando por ordem de anexo id
            linhas_correspondentes = linhas_correspondentes.sort_values(by=["processoAnexoID"], ascending=True)
            manter_id = linhas_correspondentes.iloc[0]['processoAnexoID']
        else:
            # Ordenando as linhas por data
            linhas_correspondentes = linhas_correspondentes.sort_values(by=["Publicado em"], ascending=True)
            # Mantendo a primeira linha (mais antiga)
            manter_id = linhas_correspondentes.iloc[0]['processoAnexoID']
        
        # Eliminar todas as outras com mesmo processoID e processoAnexoID diferente do que foi mantido
        sentencas_final = sentencas_final[~((sentencas_final["processoID"] == id) & (sentencas_final["processoAnexoID"] != manter_id))]

In [18]:
# Verificando dataframe após a eliminação
ids_processos = sentencas_final["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_final.loc[sentencas['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

Número de processos com mais de uma sentença: 0


In [16]:
# Verificando se a quantidade de processos após o drop se manteve
sentencas_final["processoID"].unique().shape

(223,)

In [ ]:
# Exportando df final de sentenças para um arquivo Excel
# sentencas_final.to_excel("docs/sentencas_final.xlsx", index=False)

In [29]:
def verifica_dano_ambiental(texto):
    """
    Função que verifica se o texto possui dano ambiental.
    Se houver, justifica a resposta. Caso contrário, responde "Não há dano ambiental".
    """
    
    prompt = f"""
    Você é um especialista em direito ambiental. 
    Você deve analisar o seguinte texto e verificar se há dano ambiental.
    Se houver, justifique sua resposta em até 20 palavras. Caso contrário, responda "Não há dano ambiental".
    Refira-se EXCLUSIVAMENTE ao texto fornecido para identificar se há dano ambiental e para justificar sua resposta!!

    Texto: {texto}
    """
    
    class FormatoResposta(BaseModel):
        isDanoAmbiental: bool
        justificativa: str

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': FormatoResposta,
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    time.sleep(3)
    return resposta

In [30]:
respostas = []

for id in ids_processos:
    linha = sentencas_final.loc[sentencas_final['processoID'] == id]
    texto = ''
    linha = linha.iloc[0] # extrai a linha como Series

    processoAnexoID = linha['processoAnexoID']
    link = linha['Download copia']
    response = requests.get(link)
    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    
    if 'pdf' in content_type:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto += page.get_text()
    
    elif 'html' in content_type:
        soup = BeautifulSoup(response.content, 'html.parser')
        texto += soup.get_text()

    resposta_prompt = verifica_dano_ambiental(texto)
    resposta_prompt = json.loads(resposta_prompt.text)
    resposta_prompt["processoAnexoID"] = processoAnexoID
    resposta_prompt["link_referencia"] = link
    resposta_prompt["processoID"] = id
    respostas.append(resposta_prompt)

respostas

[{'isDanoAmbiental': True,
  'justificativa': 'A empresa possuía madeira em tora sem licença válida, conforme Auto de Infração, configurando dano ambiental.',
  'processoAnexoID': np.int64(2344968081),
  'link_referencia': 'http://jud-anexos.digesto.com.br/0cacd6d80c499ae25dcb85380a07c3dd.pdf',
  'processoID': np.int64(474474022)},
 {'isDanoAmbiental': True,
  'justificativa': 'Sim, houve destruição de 121,15 hectares de floresta nativa no bioma amazônico sem autorização ambiental.',
  'processoAnexoID': np.int64(2294638461),
  'link_referencia': 'http://jud-anexos.digesto.com.br/90df1f9ac9917f6df4b9f91915b3a8bd.pdf',
  'processoID': np.int64(474474806)},
 {'isDanoAmbiental': False,
  'justificativa': 'Não há dano ambiental, pois o texto trata de uma ação de venda casada de carregadores de celular.',
  'processoAnexoID': np.int64(1511375012),
  'link_referencia': 'http://jud-anexos.digesto.com.br/c89c7a26febccd44703bb340de170f09.pdf',
  'processoID': np.int64(575718248)},
 {'isDanoAmbi

In [31]:
# Transformando a lista de respostas em um DataFrame
respostas_df = pd.DataFrame(respostas)

# Reorganizando as colunas
respostas_df = respostas_df[["processoID", "processoAnexoID", "isDanoAmbiental", "justificativa", "link_referencia"]]

respostas_df.head()

,processoID,processoAnexoID,isDanoAmbiental,justificativa,link_referencia
0,474474022,2344968081,True,A empresa possuía madeira em tora sem licença ...,http://jud-anexos.digesto.com.br/0cacd6d80c499...
1,474474806,2294638461,True,"Sim, houve destruição de 121,15 hectares de fl...",http://jud-anexos.digesto.com.br/90df1f9ac9917...
2,575718248,1511375012,False,"Não há dano ambiental, pois o texto trata de u...",http://jud-anexos.digesto.com.br/c89c7a26febcc...
3,580649179,1504080402,False,"Não há dano ambiental, pois o texto trata de i...",http://jud-anexos.digesto.com.br/585fdfebfdd6d...
4,654928926,2419043700,False,Não há dano ambiental. O texto trata de cobran...,http://jud-anexos.digesto.com.br/6be7e9beac86d...
